In [1]:
# %%
from vllm import SamplingParams, LLM
from tqdm import tqdm
from datasets import load_dataset, concatenate_datasets
from datasets import Dataset, concatenate_datasets
from datetime import datetime
import json
import os
from src.generator import prepare_records
from src.clean_utils import clean
import random
import glob
import pandas as pd
import time

/home/setup/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-24 17:27:44,737	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:

wait_time = random.randint(1, 10)
time.sleep(wait_time)

#####################
# 設定関連
n_records = 30
out_dir="0624out_data_flan"

#parquet一覧
directory_path = '/data/2024/TranslatedFlan/FLAN/'
#tubame
####################

################
#メイン

os.system(f"mkdir -p {out_dir}")

current_time_no_symbols = datetime.now().strftime(
    "%Y-%m-%d %H:%M:%S").replace("-", "").replace(":", "").replace(" ", "")
out_path = f"{out_dir}/model_{current_time_no_symbols}.jsonl"


# 再帰的にディレクトリ内のすべてのパーケットファイルを検索する
#parquet_list = glob.glob(f"{directory_path}/*/*.*.parquet", recursive=False)

# 再帰的にディレクトリ内のすべてのパーケットファイルを検索する
parquet_list = glob.glob(os.path.join(directory_path, '**', '*.parquet'), recursive=True)

In [ ]:

while True:
    try:
        parquet_path=random.choice(parquet_list)
        #ファイルサイズを確認
        file_size = os.path.getsize(parquet_path)
        if file_size <1000:
            continue
        print(f"load {parquet_path}")
        df=pd.read_parquet(parquet_path)
        ds= Dataset.from_pandas(df)
        break
    except Exception as e:
        print(e)
        time.sleep(10)
        continue



print("shuffling")
ds = ds.shuffle()

name 'random' is not defined


NameError: name 'time' is not defined

In [ ]:


print("init llm")
model_name = "microsoft/Phi-3-medium-128k-instruct"
llm = LLM(model=model_name, trust_remote_code=True,
          max_model_len=20000
          )
inst_dict = {   

"translate1":"""You are a professional translator. Translate the following Englsh into fluent Japanese.
#English""",
"translate2":"""You are a translator. Translate the following Englsh into fluent Japanese.
#English""",
"translate3":"""You are a professional translator. Translate the following Englsh into fluent Japanese.
Use formal Japanese.
#English""",
"translate4":"""You are a professional translator. Translate the following Englsh into fluent Japanese.
Use polite Japanese.
#English""",
"translate5":"""You are a professional translator. Translate the following Englsh into fluent Japanese.
Use casual Japanese.
#English""",

}              

       

mode_list = list(inst_dict.keys())


init llm


/home/setup/miniconda3/envs/llmeval/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO 06-24 17:25:16 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='microsoft/Phi-3-medium-128k-instruct', speculative_config=None, tokenizer='microsoft/Phi-3-medium-128k-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=20000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=microsoft/Phi-3-medium-128k-instruct)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 06-24 17:25:17 utils.py:660] Found nccl from library /home/setup/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 06-24 17:25:18 selector.py:27] Using FlashAttention-2 backend.
INFO 06-24 17:25:19 weight_utils.py:199] Using model weights format ['*.safetensors']
INFO 06-24 17:25:23 model_runner.py:175] Loading model weights took 26.1473 GB
INFO 06-24 17:25:26 gpu_executor.py:114] # GPU blocks: 13470, # CPU blocks: 1310
INFO 06-24 17:25:27 model_runner.py:937] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-24 17:25:27 model_runner.py:941] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-24 17:25:32 model_runner.py:1017] Graph capturing 

In [ ]:


# %%
print(len(ds), " records")
while True:

    #回答
    records = prepare_records(
        ds, mode_list, n_records=n_records,
        inst_dict=inst_dict
        
        )
    prompts = [record["original_text"] for record in records]
    outputs1 = llm.generate(
        prompts,
        sampling_params=SamplingParams(
            temperature=0.1,
            max_tokens=2048,
            repetition_penalty=1.15,
        )
    )

    for record, ja_output in zip(records, outputs1):
        ja= (ja_output.outputs[0].text).strip()

        ja=clean(ja,lang="ja")

        if ja=="":
            #print("rejected")
            #print(ja_output.outputs[0].text)
            continue


        record["ja"]=ja 
        #record.pop("original_text")


        #print("saving to "+out_path)
        with open(out_path, "a") as f:
            f.write(json.dumps(record, ensure_ascii=False) + "\n")

85978  records


Processed prompts:   0%|          | 0/31 [00:00<?, ?it/s]